In [2]:
import os
import sys
import time
import json
import argparse
import logging
from typing import List, Dict, Any, Optional
import requests
from dotenv import load_dotenv
load_dotenv()

# Get the absolute path to the directory containing the other file
other_dir = os.path.abspath('../')

# Add the directory to sys.path if it's not already there
if other_dir not in sys.path:
    sys.path.append(other_dir)

# CORE FUNCTIONS TESTS

In [3]:
GOOGLE_API_URL = "https://www.googleapis.com/customsearch/v1"
DEFAULT_NUM_RESULTS = 10
# Default delay between requests in seconds
DEFAULT_REQUEST_DELAY = 1.0

API_KEY = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
CSE_ID = os.getenv("GOOGLE_CSE_ID")

## Google Custom Engine Search function

In [4]:
from google_doc_retriever import search_google_cse

In [5]:
try:
    results_json = search_google_cse(
        query="Phó Thủ tướng Trần Hồng Hà thay mặt Chính phủ, Thủ tướng Chính phủ chúc mừng Đài Truyền hình Việt Nam, Đài truyền hình các tỉnh, thành phố trên cả nước, các đơn vị sản xuất truyền hình và TP. Hải Phòng sau 2 năm gián đoạn do đại dịch COVID-19 đã tổ chức rất thành công sự kiện quan trọng này",
        api_key=API_KEY,
        cse_id=CSE_ID
    )
except ValueError as ve:
    logging.error("Configuration error: %s", ve)
except requests.exceptions.RequestException:
        # Error already logged in search_google_cse, stop processing
    logging.error("Stopping due to network error.")

2025-04-29 15:26:02,684 - INFO - Sending request to Google CSE API for query: 'Phó Thủ tướng Trần Hồng Hà thay mặt Chính phủ, Thủ tướng Chính phủ chúc mừng Đài Truyền hình Việt Nam, Đài truyền hình các tỉnh, thành phố trên cả nước, các đơn vị sản xuất truyền hình và TP. Hải Phòng sau 2 năm gián đoạn do đại dịch COVID-19 đã tổ chức rất thành công sự kiện quan trọng này' (start: 1, num: 10)


In [6]:
results_json

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - Phó Thủ tướng Trần Hồng Hà thay mặt Chính phủ, Thủ tướng Chính phủ chúc mừng Đài Truyền hình Việt Nam, Đài truyền hình các tỉnh, thành phố trên cả nước, các đơn vị sản xuất truyền hình và TP. Hải P

## Processing function

In [7]:
from google_doc_retriever import process_search_results

In [8]:
all_results = []
total_retrieved = 0

In [9]:
processed = process_search_results(results_json)
all_results.extend(processed)
total_retrieved += len(processed)

# Check if Google indicated there are more results
# Useful if you want to stop fetching when Google runs out
search_info = results_json.get('searchInformation', {})
total_results_est = search_info.get('totalResults', '0')
logging.info(
    "Retrieved %d results for page %d. (Estimated total: %s)",
    len(processed), 1, total_results_est
)

2025-04-29 15:26:29,797 - INFO - Retrieved 10 results for page 1. (Estimated total: 414000)


In [10]:
all_results

[{'title': '5.932 tỷ đồng ủng hộ tại Chương trình phát động hỗ trợ xóa nhà tạm ...',
  'link': 'http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html',
  'snippet': 'Oct 5, 2024 ... ... Đài Tiếng nói Việt Nam và tiếp sóng trên các kênh, Đài truyền hình trong cả nước. Về việc thôi Ủy viên Ủy ban Trung ương Mặt trận Tổ quốc\xa0...'},
 {'title': "Tôn vinh Thầy thuốc Việt Nam và trao giải cuộc thi viết ''Sự hy sinh ...",
  'link': 'https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false',
  'snippet': "Feb 27, 2024 ... 20h ngày 26/2, tại Nhà hát Hồ Gươm, 40 Hàng Bài, Hà Nội diễn ra Chương trình Tôn vinh Thầy thuốc Việt Nam và trao giải cuộc thi viết 'Sự hy\xa0..."},
 {'title': 'Thủ tướng Chính phủ chỉ đạo chủ động phòng, chống rét

In [21]:
with open('retrieval_results.jsonl', 'w', encoding='utf-8') as jsonl_file:
    for result in all_results:
        jsonl_file.write(json.dumps(result, ensure_ascii=False) + '\n')

In [ ]:
test_urls = []

with open('retrieval_results.jsonl', 'r', encoding='utf-8') as jsonl_file:
    for line in jsonl_file:
        result = json.loads(line.strip())
        if 'link' in result:
            test_urls.append(result['link'])

In [14]:
test_urls

['http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html',
 'https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false',
 'https://chinhsachcuocsong.vnanet.vn/thu-tuong-chinh-phu-chi-dao-chu-dong-phong-chong-ret-dam-ret-hai-keo-dai/55191.html',
 'https://nbtv.vn/le-cong-bo-thanh-lap-thanh-pho-hoa-lu-va-cong-nhan-thanh-pho-dat-tieu-chi-do-thi-loai-i-85416.html',
 'https://hoabinhtv.vn/chinh-tri/cong-bo-quyet-dinh-bo-nhiem-2-pho-thu-tuong-tran-hong-ha-tran-luu-quang',
 'https://baothanhhoa.vn/khai-mac-lien-hoan-phat-thanh-toan-quoc-lan-thu-xvi-phat-thanh-viet-nam--da-dang-trong-chuyen-doi-so-219272.htm',
 'https://baoangiang.com.vn/thu-tuong-truyen-hinh-phai-phan-anh-sinh-dong-co-suc-lan-toa-a417491.html',
 'https://laodong.vn/thoi

In [17]:
import trafilatura

# Configure basic logging for better traceability
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define a realistic User-Agent to mimic a browser
# Some sites block default 'python-requests' user agent
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def extract_main_text_from_url(url: str, timeout: int = 15) -> Optional[str]:
    """
    Fetches content from a given URL and extracts the main text content.

    Uses 'requests' to fetch the HTML and 'trafilatura' to extract the
    main article text, attempting to remove boilerplate (nav, ads, footer, etc.).

    Args:
        url: The URL of the web page to process.
        timeout: The timeout in seconds for the HTTP request. Defaults to 15.

    Returns:
        The extracted main text content as a string, or None if an error
        occurs during fetching or extraction, or if no main content is found.
    """
    logging.info(f"Attempting to fetch and extract main text from: {url}")

    try:
        # 1. Fetch the HTML content
        response = requests.get(url, headers=HEADERS, timeout=timeout, allow_redirects=True)
        
        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status()

        # Check if the content type seems to be HTML
        content_type = response.headers.get('content-type', '').lower()
        if 'html' not in content_type:
            logging.warning(f"Content type '{content_type}' for URL {url} doesn't appear to be HTML. Skipping extraction.")
            return None

        html_content = response.text

    except requests.exceptions.Timeout:
        logging.error(f"Timeout occurred while fetching URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching URL {url}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred during fetch for {url}: {e}")
        return None

    if not html_content:
         logging.warning(f"No HTML content retrieved from {url}")
         return None

    try:
        # 2. Extract the main text using trafilatura
        # include_comments=False is often desirable
        # target_language can help if you know the expected language
        main_text = trafilatura.extract(
            html_content,
            include_comments=False,
            include_tables=True, # Set to False if tables are usually boilerplate
            output_format='txt', # Options: 'txt', 'xml', 'json'
            include_images=False, # Optional: disable image download
            include_formatting=False, # Optional: disable formatting
        )

        if main_text:
            logging.info(f"Successfully extracted main text from: {url}")
            return main_text.strip() # Remove leading/trailing whitespace
        else:
            logging.warning(f"Trafilatura could not extract main text from: {url}. The page might lack a clear article structure or be JavaScript-heavy.")
            return None

    except Exception as e:
        # Catch potential errors during trafilatura processing
        logging.error(f"Error during main text extraction for {url}: {e}")
        return None

In [20]:
print("--- Starting URL Processing ---")
for target_url in test_urls:
    print(f"\n>>> Processing URL: {target_url}")
    extracted_content = extract_main_text_from_url(target_url)

    if extracted_content:
        print("-" * 20)
        print(f"Extracted Content (First 500 chars):\n{extracted_content[:500]}...")
        print("-" * 20)
    else:
        print("Could not extract main content.")
    print("-" * 50)

print("\n--- URL Processing Finished ---")

2025-04-29 16:14:42,991 - INFO - Attempting to fetch and extract main text from: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html


--- Starting URL Processing ---

>>> Processing URL: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html


2025-04-29 16:14:43,293 - INFO - Successfully extracted main text from: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html
2025-04-29 16:14:43,293 - INFO - Attempting to fetch and extract main text from: https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false
2025-04-29 16:14:43,393 - ERROR - Error fetching URL https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false: HTTPSConnectionPool(host='moh.gov.vn', port=443): Max retries exceeded with url: /hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-l

--------------------
Extracted Content (First 500 chars):
|
| Thủ tướng Chính phủ Phạm Minh Chính cùng các đồng chí lãnh đạo Đảng, Nhà nước, MTTQ Việt Nam tham dự chương trình |
Tham dự chương trình có Thủ tướng Chính phủ Phạm Minh Chính; ông Lương Cường, Ủy viên Bộ Chính trị, Thường trực Ban Bí thư; ông Đỗ Văn Chiến, Ủy viên Bộ Chính trị, Bí thư Trung ương Đảng, Chủ tịch Ủy ban Trung ương MTTQ Việt Nam; ông Nguyễn Trọng Nghĩa, Ủy viên Bộ Chính trị, Bí thư Trung ương Đảng, Trưởng Ban Tuyên giáo Trung ương; ông Nguyễn Hòa Bình, Ủy viên Bộ Chính trị, Phó...
--------------------
--------------------------------------------------

>>> Processing URL: https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false
Could not extract main content.
--------------------------------------------------

>>> Processing URL: https://chinhsachcuocsong.vnanet.vn/thu-tuong-c

2025-04-29 16:14:43,491 - INFO - Attempting to fetch and extract main text from: https://nbtv.vn/le-cong-bo-thanh-lap-thanh-pho-hoa-lu-va-cong-nhan-thanh-pho-dat-tieu-chi-do-thi-loai-i-85416.html


...

--------------------
Extracted Content (First 500 chars):
Thủ tướng Phạm Minh Chính phát biểu tại Lễ bế mạc Liên hoan Truyền hình toàn quốc lần thứ 42. (Ảnh: Dương Giang/TTXVN)
Tối 22/3, Thủ tướng Chính phủ Phạm Minh Chính dự Lễ bế mạc và trao giải Liên hoan Truyền hình toàn quốc lần thứ 42 tại thành phố Quy Nhơn, tỉnh Bình Định.
Phát biểu tại sự kiện, Thủ tướng Chính phủ mong muốn những người làm truyền hình tạo ra nhiều chương trình hấp dẫn, có chiều sâu, mang giá trị nhân văn, phản ánh sinh động hơi thở, nhịp đập của cuộc sống; có sức lan tỏa và tru...
--------------------
--------------------------------------------------

>>> Processing URL: https://laodong.vn/thoi-su/phan-cong-cong-tac-thu-tuong-va-7-pho-thu-tuong-chinh-phu-1468325.ldo
Could not extract main content.
--------------------------------------------------

>>> Processing URL: https://baochinhphu.vn/thuc-day-hop-tac-viet-nam-va-hoa-ky-trong-cac-linh-vuc-moi-noi-102241031171910482.htm


2025-04-29 16:14:45,298 - INFO - Successfully extracted main text from: https://baochinhphu.vn/thuc-day-hop-tac-viet-nam-va-hoa-ky-trong-cac-linh-vuc-moi-noi-102241031171910482.htm
2025-04-29 16:14:45,298 - INFO - Attempting to fetch and extract main text from: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


--------------------
Extracted Content (First 500 chars):
Tải ứng dụng:
BÁO ĐIỆN TỬ CHÍNH PHỦ
Tại buổi tiếp, Phó Thủ tướng Trần Hồng Hà đánh giá cao những đóng góp của Đại sứ Marc Knapper trong việc thúc đẩy quan hệ hai nước, đặc biệt là những thành tựu mà hai nước đã đạt được sau một năm nâng cấp quan hệ lên Đối tác Chiến lược toàn diện.
Phó Thủ tướng khẳng định, Việt Nam coi Hoa Kỳ là đối tác có tầm quan trọng chiến lược và mong muốn cùng Hoa Kỳ tiếp tục đưa khuôn khổ quan hệ mới đi vào chiều sâu, hiệu quả, thực chất trên cơ sở tôn trọng độc lập, chủ...
--------------------
--------------------------------------------------

>>> Processing URL: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


2025-04-29 16:14:46,288 - INFO - Successfully extracted main text from: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


--------------------
Extracted Content (First 500 chars):
Lễ công bố quyết định của Thủ tướng Chính phủ chuyển Trường Đại học Kinh tế Thành phố Hồ Chí Minh thành Đại học Kinh tế Thành phố Hồ Chí Minh
18 tháng 11 năm 2023
Ngày 18 tháng 11 năm 2023, “Lễ Công bố Quyết định của Thủ tướng Chính phủ chuyển Trường Đại học Kinh tế Thành phố Hồ Chí Minh thành Đại học Kinh tế Thành phố Hồ Chí Minh” đã chính thức diễn ra, đánh dấu một cột mốc mới đối với lịch sử Đại học Kinh tế TP. Hồ Chí Minh (UEH) và ngành giáo dục Việt Nam. Từ đây, UEH sẵn sàng cho một hành tr...
--------------------
--------------------------------------------------

--- URL Processing Finished ---


In [22]:
# Define the output JSONL file name
output_filename = "extracted_content.jsonl"

print(f"--- Starting URL Processing ---")
print(f"Results will be appended to: {output_filename}")

# Open the file in append mode ('a') with UTF-8 encoding
# Use 'w' if you want to overwrite the file on each run
try:
    with open(output_filename, 'a', encoding='utf-8') as outfile:
        for target_url in test_urls:
            print(f"\n>>> Processing URL: {target_url}")
            extracted_content = extract_main_text_from_url(target_url)

            # Prepare the data record for JSON output
            # The value for 'text' will be the extracted string or None
            data_record = {
                "url": target_url,
                "text": extracted_content
            }

            # Convert the dictionary to a JSON string
            # ensure_ascii=False preserves non-ASCII characters correctly
            json_line = json.dumps(data_record, ensure_ascii=False)

            # Write the JSON string as a line in the output file
            outfile.write(json_line + '\n')

            if extracted_content:
                print(f"Successfully extracted and saved data for {target_url}")
            else:
                print(f"Failed to extract text for {target_url}, saved record with null text.")
            print("-" * 50)

except IOError as e:
    print(f"\nError: Could not write to file {output_filename}. Reason: {e}")
except Exception as e:
    print(f"\nAn unexpected error occurred during file writing: {e}")


print(f"\n--- URL Processing Finished ---")

2025-04-29 16:17:42,768 - INFO - Attempting to fetch and extract main text from: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html


--- Starting URL Processing ---
Results will be appended to: extracted_content.jsonl

>>> Processing URL: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html


2025-04-29 16:17:43,124 - INFO - Successfully extracted main text from: http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html
2025-04-29 16:17:43,127 - INFO - Attempting to fetch and extract main text from: https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false
2025-04-29 16:17:43,252 - ERROR - Error fetching URL https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false: HTTPSConnectionPool(host='moh.gov.vn', port=443): Max retries exceeded with url: /hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-l

Successfully extracted and saved data for http://mattran.org.vn/hoat-dong/5932-ty-dong-ung-ho-tai-chuong-trinh-phat-dong-ho-tro-xoa-nha-tam-nha-dot-nat-cho-ho-ngheo-ho-co-hoan-canh-kho-khan-tren-pham-vi-ca-nuoc-57312.html
--------------------------------------------------

>>> Processing URL: https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false
Failed to extract text for https://moh.gov.vn/hoat-dong-cua-lanh-dao-bo/-/asset_publisher/TW6LTp1ZtwaN/content/ton-vinh-thay-thuoc-viet-nam-va-trao-giai-cuoc-thi-viet-su-hy-sinh-tham-lang-lan-thu-vi?inheritRedirect=false, saved record with null text.
--------------------------------------------------

>>> Processing URL: https://chinhsachcuocsong.vnanet.vn/thu-tuong-chinh-phu-chi-dao-chu-dong-phong-chong-ret-dam-ret-hai-keo-dai/55191.html


2025-04-29 16:17:43,307 - INFO - Attempting to fetch and extract main text from: https://nbtv.vn/le-cong-bo-thanh-lap-thanh-pho-hoa-lu-va-cong-nhan-thanh-pho-dat-tieu-chi-do-thi-loai-i-85416.html


...

Successfully extracted and saved data for https://baoangiang.com.vn/thu-tuong-truyen-hinh-phai-phan-anh-sinh-dong-co-suc-lan-toa-a417491.html
--------------------------------------------------

>>> Processing URL: https://laodong.vn/thoi-su/phan-cong-cong-tac-thu-tuong-va-7-pho-thu-tuong-chinh-phu-1468325.ldo
Failed to extract text for https://laodong.vn/thoi-su/phan-cong-cong-tac-thu-tuong-va-7-pho-thu-tuong-chinh-phu-1468325.ldo, saved record with null text.
--------------------------------------------------

>>> Processing URL: https://baochinhphu.vn/thuc-day-hop-tac-viet-nam-va-hoa-ky-trong-cac-linh-vuc-moi-noi-102241031171910482.htm


2025-04-29 16:17:45,130 - INFO - Successfully extracted main text from: https://baochinhphu.vn/thuc-day-hop-tac-viet-nam-va-hoa-ky-trong-cac-linh-vuc-moi-noi-102241031171910482.htm
2025-04-29 16:17:45,130 - INFO - Attempting to fetch and extract main text from: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


Successfully extracted and saved data for https://baochinhphu.vn/thuc-day-hop-tac-viet-nam-va-hoa-ky-trong-cac-linh-vuc-moi-noi-102241031171910482.htm
--------------------------------------------------

>>> Processing URL: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


2025-04-29 16:17:46,147 - INFO - Successfully extracted main text from: https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085


Successfully extracted and saved data for https://ueh.edu.vn/cuoc-song-ueh/tin-tuc/le-cong-bo-quyet-dinh-cua-thu-tuong-chinh-phu-chuyen-truong-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-thanh-dai-hoc-kinh-te-thanh-pho-ho-chi-minh-71085
--------------------------------------------------

--- URL Processing Finished ---
